In [8]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import re
import random
import MeCab
import jieba
import numpy as np
from six.moves import xrange
import tensorflow.compat.v1 as tf

# 停用词

In [9]:
stop_words = []
with open('ja_stopwords.txt','r',encoding='utf-8')as f_stopwords:
    for line in f_stopwords:
        line  =line.replace('\r','').replace('\n','').strip()
        stop_words.append(line)

stop_words = set(stop_words)  #去重
print(len(stop_words))

142


# 预处理

In [10]:
tagger_owakati = MeCab.Tagger('-Owakati')
raw_word_list = []
sentence_owakati = []
rules = u'([\u3040-\u9faf]+)'
pattern = re.compile(rules)#正则
count = 0
#f_writer = open('挪威的森林_pro.txt','w',encoding = 'utf-8')
with open('挪威的森林.txt','r',encoding = 'utf-8')as f_reader:
    lines = f_reader
    for line in lines:
        line = line.replace('\r',' ').replace('\n',' ').strip()
        if line == '' or line is None:
            continue
        line = ''.join(tagger_owakati.parse(line))
        seg_list = pattern.findall(line)
        sentence_owakati.append(seg_list)
        word_list = []
        for word in seg_list:
            if word not in stop_words:
                word_list.append(word)
        if len(word_list)>0:
            raw_word_list.extend(word_list)
            line = ' '.join(word_list)
  #          f_writer.write(line+'\n')
 #           f_writer.flush()

#f_writer.close()
print(raw_word_list[:30])
print(len(raw_word_list))
print(len(set(raw_word_list)))
print(sentence_owakati[:10])
vocabulary_size = len(set(raw_word_list))
#     for line in f_reader:
#         line = line.replace('\r','').replace('\n','').strip()
        
            

['ノルウェー', '森', '村上', '春樹', '第', '一', '章', '僕', '三十', '七', '歳', 'ボーイング', 'シート', '座っ', '巨大', '飛行', '機', 'ぶ厚い', '雨雲', 'くぐり抜け', '降下', 'ハンブルク', '空港', '着陸', 'しよう', '十', '一', '月', '冷ややか', '雨']
76346
8875
[['ノルウェー', 'の', '森'], ['村上', '春樹'], ['第', '一', '章'], ['僕', 'は', '三十', '七', '歳', 'で', 'その', 'とき', 'ボーイング', 'の', 'シート', 'に', '座っ', 'て', 'い', 'た', 'その', '巨大', 'な', '飛行', '機', 'は', 'ぶ厚い', '雨雲', 'を', 'くぐり抜け', 'て', '降下', 'し', 'ハンブルク', '空港', 'に', '着陸', 'しよう', 'と', 'し', 'て', 'いる', 'ところ', 'だっ', 'た', '十', '一', '月', 'の', '冷ややか', 'な', '雨', 'が', '大地', 'を', '暗く', '染め', '雨合羽', 'を', '着', 'た', '整備', '工', 'たち', 'や', 'のっぺり', 'と', 'し', 'た', '空港', 'ビル', 'の', '上', 'に', '立っ', 'た', '旗', 'や', 'の', '広告', '板', 'や', 'そんな', '何', 'も', 'か', 'も', 'を', 'フランドル', '派', 'の', '陰うつ', 'な', '絵', 'の', '背景', 'の', 'よう', 'に', '見せ', 'て', 'い', 'た', 'やれ', 'やれ', 'また', 'ドイツ', 'か', 'と', '僕', 'は', '思っ', 'た'], ['飛行', '機', 'が', '着地', 'を', '完了', 'する', 'と', '禁煙', 'の', 'サイン', 'が', '消え', '天井', 'の', 'スピーカー', 'から', '小さな', '音', 'で', 'が', '流れ', 'はじめ', '

# 文本编码

In [11]:
words = raw_word_list
count = [['UNK','-1']]
collections.Counter(words).most_common(20)
count.extend(collections.Counter(words).most_common(vocabulary_size))
#print(count)

dictionary = dict()   #创建一个按照出现个数逆序编码 用字符对应整形
for word , _ in count:
    dictionary[word] = len(dictionary) 

#print(dictionary)

data = list()
unk_count = 0
for word in words:       #用字典编码表示真个文本 存入data列表
    if word in dictionary:
        index = dictionary[word]
    else:
        index = 0      #如果出现未知则用0标识
        unk_count +=1
    data.append(index)

count[0][1] = unk_count #最后将未知的个数输入

#颠倒键值对 用整数查找字符
reverse_dictionary = dict(zip(dictionary.values(),dictionary.keys()))
reverse_dictionary[11]



'あなた'

# 批量处理生成

In [12]:
#data_index = 0
# batch_size = 8
# num_skips = 2  # 同时取词的个数
# skip_window = 1 #窗口大小
# batch = np.ndarray(shape = (batch_size),dtype = np.int32)
# labels = np.ndarray(shape = (batch_size,1),dtype = np.int32)
# span = 2*skip_window +1
# buffer = collections.deque(maxlen = span)
# for _ in range(span):
#     buffer.append(data[data_index])
#     data_index = (data_index +1) %len(data)
# for i in range(batch_size // num_skips):
#     target = skip_window
#     targets_to_avoid = [skip_window]
#     for j in range(num_skips):
#         while target in targets_to_avoid:
#             target = random.randint(0,span-1)
#         targets_to_avoid.append(target)
#         batch[i * num_skips +j ] = buffer[skip_window]
#         labels[ i * num_skips +j] = buffer[target]
#     buffer.append(data[data_index])
#     data_index = (data_index +1)% len(data)

In [13]:
# for i in range(8):
#     print (batch[i],reverse_dictionary[batch[i]],'-->',labels[i,0],reverse_dictionary[labels[i,0]])

# 封装

In [14]:
data_index = 0

def generate_batch(batch_size,num_skips,skip_window):
    global data_index
    batch = np.ndarray(shape = (batch_size),dtype = np.int32)
    labels = np.ndarray(shape = (batch_size,1),dtype = np.int32)
    span = 2*skip_window +1
    buffer = collections.deque(maxlen = span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index +1) %len(data)
    for i in range(batch_size // num_skips):
        target = skip_window
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0,span-1)
            targets_to_avoid.append(target)
            batch[i * num_skips +j ] = buffer[skip_window]
            labels[ i * num_skips +j] = buffer[target]
        buffer.append(data[data_index]) 
        data_index = (data_index +1)% len(data)
    return batch,labels
    

In [15]:
batch,labels = generate_batch(batch_size = 128, num_skips = 4, skip_window =2)

for i in range(120):
    print (batch[i],reverse_dictionary[batch[i]],'-->',labels[i,0],reverse_dictionary[labels[i,0]])

# skipgram model

In [41]:
batch_size = 128
embedding_size = 300
skip_window = 2
num_skips = 4
valid_window = 100
num_sample = 64

learning_rate = 0.01

#校验集Z
valid_word = ['緑','僕','永沢']
valid_examples = [dictionary[li] for li in valid_word]

graph = tf.Graph()
with graph.as_default():
    #输入
    train_inputs = tf.placeholder(tf.int32,shape=[batch_size]) #训练集
    train_labels = tf.placeholder(tf.int32,shape=[batch_size,1])#labels 
    valid_dataset = tf.constant(valid_examples,dtype=tf.int32)#检验
    with tf.device('/cpu:0'):
        embeddings = tf.Variable(tf.random_uniform([vocabulary_size,embedding_size],-1.0,1.0))
        embed = tf.nn.embedding_lookup(embeddings,train_inputs)
        
        nce_weight = tf.Variable(tf.truncated_normal([vocabulary_size,embedding_size],
                                                     stddev=1.0/math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]),
                                 dtype = tf.float32)
        
        neg_loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weight,
                                                 biases = nce_biases,
                                                 inputs = embed,
                                                 labels= train_labels,
                                                num_sampled = num_sample,
                                                num_classes = vocabulary_size))
    
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(neg_loss)
        
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings),1,keep_dims=True))
        normalized_embeddings = embeddings / norm
        
        valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings,valid_dataset)
        similarity = tf.matmul(valid_embeddings,normalized_embeddings,transpose_b=True)
        init = tf.global_variables_initializer()
    

# 模型训练

In [44]:
num_steps = 20000
with tf.Session(graph = graph) as sess:
    init.run()
    print('======initialization complete======')
    
    avg_loss = 0
    
    for step in xrange(num_steps):
        batch_inputs,batch_labels = generate_batch(batch_size,
                                                   num_skips,
                                                   skip_window)
        _, loss = sess.run([optimizer,neg_loss],feed_dict = {
            train_inputs:batch_inputs,
            train_labels:batch_labels
        })
        avg_loss = avg_loss + loss
        
        if step % 5000 == 0:
            if step > 0:
                avg_loss = avg_loss / 5000
            print("average loss is among",step,"is",avg_loss)
            
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(len(valid_word)):
                val_word = reverse_dictionary[valid_examples[i]]
                top_k = 10
                nearest = (sim[i,:].argsort()[:top_k])
                print(nearest)
                sim_str = 'the top 10 nearest words to',val_word,'are:'
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    sim_str = '%s %s' %(sim_str,close_word)
                print(sim_str)
    final_embeddings = normalized_embeddings.eval()

======initialization complete======
average loss is among 0 is 216.22235107421875
[1185 1042 7923 2644 1643 7157 2942 2881 4938 6461]
('the top 10 nearest words to', '緑', 'are:') 電気 閉め 鰻屋 無力 一人ぼっち バッチリ 薄 動詞 すえ しま
[ 991 4722 7204  564 8859 8826 3023 4118 6540 6548]
('the top 10 nearest words to', '僕', 'are:') おなか 過ぎ去っ もがく 撫で 興 恋愛 ぜんぜん 食料 はずし 殺し
[7915 1542 7098  695 7661 7376 6058  869 7103   71]
('the top 10 nearest words to', '永沢', 'are:') うーん 一杯 幾 ニ 高熱 ねっ 地底 飲ま 本能 いっ


InvalidArgumentError: Graph execution error:

Detected at node 'embedding_lookup' defined at (most recent call last):
    File "C:\Users\fakenews\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\fakenews\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\fakenews\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\fakenews\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 985, in launch_instance
      app.start()
    File "C:\Users\fakenews\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\fakenews\AppData\Local\Programs\Python\Python310\lib\site-packages\tornado\p